In [1]:
import pandas as pd
import numpy as np
import pylab as pl
import scipy.io as sio
import seaborn as sns
import statsmodels.api as sm
import funcs as func
import matplotlib.pyplot as plt
import sys
import pdb
import glob
import pickle

import sys
sys.path.append('/home/jyotika/Utils/hddm/')

import hddm

data_dir = "./Data/Y_maze/Decision_times/"
data_target_dir = "./Data/processed_data/Y_maze/"
figure_dir = "./Figures/Y_maze/"

plt.rcParams["figure.facecolor"] = "w"

In [2]:
all_conflicts = pd.read_csv(data_target_dir+"all_experiments_df_with_DTs.csv")

In [4]:
all_conflicts

,Unnamed: 0,conflict,condition,volatility,event_marker,block,reward_probability,trial_length_range,session,block_num,trial_num,chosen_action,rewarded,optimal_action,RT(ms),conflict + volatility
0,0,Low,Control,High,8,left,10-90,10-20,0,1,0,right,unrewarded,0.0,606,Low - High
1,1,Low,Control,High,8,left,10-90,10-20,0,1,0,right,unrewarded,0.0,844,Low - High
2,2,Low,Control,High,8,left,10-90,10-20,0,1,0,right,unrewarded,0.0,1331,Low - High
3,3,Low,Control,High,8,left,10-90,10-20,0,1,0,right,unrewarded,0.0,667,Low - High
4,4,Low,Control,High,8,left,10-90,10-20,0,1,0,right,unrewarded,0.0,244,Low - High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46575207,7513553,No,Control,High,6,right,0-90,10-20,11,46,4,right,unrewarded,1.0,1348,No - High
46575208,7513554,No,Control,High,6,right,0-90,10-20,11,46,4,right,unrewarded,1.0,457,No - High
46575209,7513555,No,Control,High,6,right,0-90,10-20,11,46,4,right,unrewarded,1.0,1369,No - High
46575210,7513556,No,Control,High,6,right,0-90,10-20,11,46,4,right,unrewarded,1.0,1055,No - High


In [5]:
all_conflicts['stim'] = all_conflicts["conflict + volatility"].copy()

In [6]:
seed = np.random.randint(0,99999999,1)[0]
print(seed)
np.random.seed(seed)


n_samples,n_burn,n_thin = 10000, 1000, 0
n_effective_samples = n_samples - n_burn

hddm_data = pd.DataFrame()

# Takes forever with the whole data, hence trying with subset of the data
for grp in all_conflicts.groupby(["conflict","volatility"]):
    # Randomly choose 1 sessions and 5 consecutive trials 
    num_sessions = np.unique(grp[1]["session"])
    print(num_sessions)
    ind = np.random.choice(num_sessions,1)
    ind_trials = np.unique(grp[1].loc[grp[1]["session"]==ind[0]]["trial_num"])
    rand_trial = np.random.choice(ind_trials,1)[0]
    ind_trials.sort()
    print(ind_trials)
    ind_trials = ind_trials[rand_trial:rand_trial+6]
    temp = pd.DataFrame()
    for x in ind:
        for t in ind:
            temp = temp.append(grp[1].loc[(grp[1]["session"]==x)&(grp[1]["trial_num"]==t)])
    #temp = grp[1].iloc[ind][["optimal_action","RT(ms)","stim"]].copy()
    hddm_data = hddm_data.append(temp)

hddm_data["RT(ms)"] = hddm_data["RT(ms)"]/1000.
hddm_data = hddm_data.rename(columns={'optimal_action':'response','RT(ms)':'rt'})
    
print(len(hddm_data))    
m = hddm.HDDM(hddm_data,depends_on={'v':'stim','a':'stim'})

    
    

83712455
[0 1 2 3 4 5 6 7 8 9]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38]
[ 0  1  2  3  4  5  6  7  8  9 10 11]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40]
[ 0  1  2  3  4  5  6  7  8  9 10 11]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28]
179236
No model attribute --> setting up standard HDDM
Set model to ddm


In [7]:
m.find_starting_values()



-213748.4306544437
-213747.69271535455


In [ ]:
m.sample(n_samples,n_burn)
stats = m.gen_stats()

 [-------------    36%                  ] 3614 of 10000 complete in 1581.3 sec

In [ ]:
m.gen_stats()

In [ ]:
v_HH, v_HL, v_LH,v_LL, v_NH,v_NL = m.nodes_db.node[['v(High - High)', 'v(High - Low)', 'v(Low - High)','v(Low - Low)','v(No - High)','v(No - Low)']]

In [ ]:
a_HH, a_HL, a_LH,a_LL, a_NH,a_NL = m.nodes_db.node[['a(High - High)', 'a(High - Low)', 'a(Low - High)','a(Low - Low)','a(No - High)','a(No - Low)']]

In [ ]:
hddm.analyze.figure(figsize=(10,10))
hddm.analyze.plot_posterior_nodes([v_HH, v_HL, v_LH,v_LL, v_NH,v_NL])
pl.savefig(figure_dir+"drift_rates_vs_conflict_volatility_"+str(seed)+".png")

In [ ]:
hddm.analyze.plot_posterior_nodes([a_HH, a_HL, a_LH,a_LL, a_NH,a_NL])
pl.savefig(figure_dir+"thresholds_vs_conflict_volatility_"+str(seed)+".png")

In [ ]:
# m.plot_posteriors()

In [ ]:
models = dict()
models['stats'] = m.gen_stats()
models['traces'] = m.get_traces()
models['model'] = m.get_average_model()


pickle.dump(models,open(data_target_dir+"model_ddm_"+str(seed)+'.pickle','wb'))

In [ ]:
m.get_traces()

In [ ]:
m.

In [ ]:
m1 = pickle.load(open(data_target_dir+"model_ddm_"+str(seed)+'.pickle','rb'))

In [ ]:
m1